In [ ]:
import os
import json
import pandas as pd
import traceback

In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

In [ ]:
key=os.getenv("OPENAI_API_KEY")
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.5)

In [ ]:
llm # Model is working

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2
load_dotenv()


In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [ ]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {numbers} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check allthe questions to be conforming the text as well.
Make sure the format of your response is like RESPONSE_JSON below and use it as a guide.
Ensure to make {numbers} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
quiz_generator_prompt = PromptTemplate(
    input_variables=["text", "numbers","subject","tone","response_json"],
    template=TEMPLATE
)

In [ ]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generator_prompt,
    output_key="quiz",
    verbose=True
)
# Since I am using hpt 2 via hugging face so I am not using openAI API 
# therefore the error given below is for version compatibility for OpenAI

In [ ]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [ ]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [ ]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "numbers", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [ ]:
file_path=r"C:\Users\HP\mcqgen\data.txt"

In [ ]:
with open(file_path,'r') as f:
    data=f.read()

In [ ]:
print(data)

In [ ]:
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [ ]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": data,
            "numbers": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
response

In [ ]:
quiz=response.get("quiz")

In [ ]:
import re

# Extract JSON substring from quiz string
json_match = re.search(r'(\{.*\})', quiz, re.DOTALL)
if json_match:
	quiz_json_str = json_match.group(1)
	quiz = json.loads(quiz_json_str)
else:
	raise ValueError("No valid JSON found in quiz string")

In [ ]:
quiz_table_data = []
mcq = quiz["mcq"]
options = " | ".join(
    [f"{option}: {option_value}" for option, option_value in quiz["options"].items()]
)
correct = quiz["correct"]
quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')